In [ ]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [ ]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [ ]:
# Leave this code as-is

POS_NOUN = 'NN'
POS_VERB = 'VB'
POS_ADJECTIVE = 'JJ'

# Producer in Python that reads a file by words that are nouns
def read_by_parts_of_speech(filename, parts_of_speech, max_words=-1, report_every=-1):

    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:

        # Initialize counter of words to stop at max_words
        counter = 0

        # Iterate through lines in the file
        for line in file:

            elements = line.split("\t")

            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()

            if counter > max_words and max_words != -1:
                break

            for sentence in nltk.sent_tokenize(text):

                tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
                for word in [part[0] for part in tagged if part[1] in parts_of_speech]:

                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=30000, report_every=10000):
    # Prints 1/1000 of words
    if random.random() < 0.001:
        print("Current noun '%s'" % (word))

Current noun 'greasy'
Current noun 'shoot'
Current noun 'same'
Current noun 'chief'
Current noun 'important'
Current noun 'few'
Current noun 'own'
Current noun 'incept'
Current noun 'rude'
Current noun 'hungry'
- Read 10000/30000 words so far
Current noun 'deep'
Current noun 'interested'
Current noun 'old'
Current noun 'welcome'
Current noun 'american'
Current noun 'hard'
Current noun 'goddamn'
Current noun 'proper'
Current noun 'mexican'
Current noun 'astonishing'
Current noun 'fuckin'
Current noun 'first'
Current noun 'exact'
- Read 20000/30000 words so far
Current noun 'good'
Current noun 'noble'
Current noun 'sure'
Current noun 'educational'
Current noun 'terrible'
Current noun 'stupid'
Current noun 'warm'
Current noun 'wrong'
Current noun 'new'
Current noun 'nice'
Current noun 'erudite'
Current noun 'well-'
Current noun 'good'
- Read 30000/30000 words so far


# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [ ]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        if random.random() < max_reservoir_size / (len(reservoir) + 1):
            evict_index = random.randint(0, len(reservoir) - 1)
            reservoir[evict_index] = item

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [ ]:
def reservoir_sampling(filename, POS, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    words_read = 0

    for word in read_by_parts_of_speech(filename, POS, max_words=max_words, report_every=report_every):
        add_to_reservoir(reservoir, word, reservoir_size)
        words_read += 1

    return (words_read, reservoir)

In [ ]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=30000, report_every=10000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 1500


In [ ]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

54 good
33 right
32 other
32 little
32 dead
27 big
25 sure
22 real
21 old
20 last
18 wrong
18 sorry
18 much
15 whole
14 next
14 great
14 first
14 few
13 ready
12 long


<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [ ]:
# Compute relative frequencies
total_items_in_reservoir = len(reservoir)
print("Top 20 most frequent items and their relative frequencies:")

for absolute_frequency, word in most_frequent_items:
    relative_frequency = (absolute_frequency / total_items_in_reservoir) * 100
    print(f"{absolute_frequency} ({relative_frequency:.2f}%) {word}")

Top 20 most frequent items and their relative frequencies:
54 (3.60%) good
33 (2.20%) right
32 (2.13%) other
32 (2.13%) little
32 (2.13%) dead
27 (1.80%) big
25 (1.67%) sure
22 (1.47%) real
21 (1.40%) old
20 (1.33%) last
18 (1.20%) wrong
18 (1.20%) sorry
18 (1.20%) much
15 (1.00%) whole
14 (0.93%) next
14 (0.93%) great
14 (0.93%) first
14 (0.93%) few
13 (0.87%) ready
12 (0.80%) long


<font size="+1" color="red">Increase the max limit of words so that one pass takes about 2-3 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [ ]:
def estimate_frequencies_for_reservoir(filename, reservoir_size, max_words=-1, report_every=10000, top_n=5):
        # Perform reservoir sampling
        words_read, reservoir = reservoir_sampling(filename, [POS_ADJECTIVE], reservoir_size, max_words=max_words, report_every=report_every)

        # Compute frequencies in the reservoir
        freq = {}
        for item in reservoir:
            freq[item] = reservoir.count(item)

        # Compute estimates for the dataset
        estimates = [
            (word, count, count * len(reservoir) / reservoir_size)
            for word, count in freq.items()
        ]

        # Sort by estimated frequency
        estimates.sort(key=lambda x: x[2], reverse=True)

        # Print results
        print(f"Reservoir size: {reservoir_size}")
        print(f"Top {top_n} words and their frequency estimates:")
        for word, count_in_reservoir, estimated_absolute_frequency in estimates[:top_n]:
            estimated_relative_frequency = (estimated_absolute_frequency / len(reservoir)) * 100
            print(f"{word}: Count in reservoir = {count_in_reservoir}, Estimated absolute frequency = {estimated_absolute_frequency:.2f}, Estimated relative frequency = {estimated_relative_frequency:.2f}%")

In [ ]:
reservoir_sizes = [50, 100, 500, 1000, 5000]

# Run the function to compute frequency estimates
for reservoir_size in reservoir_sizes:
  estimate_frequencies_for_reservoir(INPUT_FILE, reservoir_size, max_words=30000)
  print()

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Reservoir size: 50
Top 5 words and their frequency estimates:
little: Count in reservoir = 3, Estimated absolute frequency = 3.00, Estimated relative frequency = 6.00%
sure: Count in reservoir = 3, Estimated absolute frequency = 3.00, Estimated relative frequency = 6.00%
good: Count in reservoir = 2, Estimated absolute frequency = 2.00, Estimated relative frequency = 4.00%
tough: Count in reservoir = 2, Estimated absolute frequency = 2.00, Estimated relative frequency = 4.00%
first: Count in reservoir = 2, Estimated absolute frequency = 2.00, Estimated relative frequency = 4.00%

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Reservoir size: 100
Top 5 words and their frequency estimates:
good: Count in reservoir = 5, Estimated absolute frequency = 5.00, Estimated relative frequency = 5.00%
big: Count in reservoir = 3, Estimated absolute freque

<font size="+1" color="red">Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

We can see there is a big difference of top-5 between reservoir_size=50 and reservoir_size=100. Between reservoir_size=100 and reservoir_size=500, we can see one match 'good' but there is not enough matches to say reservoir_size=100 es the best option.

Between reservoir_size=500 and reservoir_size=1000 we can see four matches 'good', 'right', 'big' and 'sure'. So, reservoir_size=500can be a good candidate.

Between reservoir_size=1000 and reservoir_size=5000 we just see two matches 'good' and 'right'. There is not a lof of matchese to take into account.

To conclude, the best reservoir size is 500 due to amount matches between reservoir_size=500 and reservoir_size=1000.

# 2. Determine approximately the distinct number of words

In [ ]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [ ]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [ ]:
'''with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
              for line in file:
                  for word in line.split():'''

'with gzip.open(INPUT_FILE, "rt", encoding=\'utf8\') as file:\n              for line in file:\n                  for word in line.split():'

In [ ]:
def calculate_estimates(POS, filename=INPUT_FILE, max_words=30000, number_of_passes=5, report_every=10000):
  estimates = []
  for i in range(number_of_passes):
      # Generate a new random hash function for this pass
          hash_function = random_hash_function()

          # Initialize maximum trailing zeroes seen
          max_trailing_zeroes = 0

          # Read the file and process each word
          for word in read_by_parts_of_speech(filename, POS, max_words=max_words, report_every=report_every):
              # Compute the hash value and count trailing zeroes
              hash_value = hash_function(word)
              trailing_zeroes = count_trailing_zeroes(abs(hash_value))
              max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

          # Estimate based on the maximum trailing zeroes
          estimate = 2 ** max_trailing_zeroes
          estimates.append(estimate)
          print(f"Estimate on pass {i + 1}: {estimate} distinct words")
  return estimates

In [ ]:
estimates=calculate_estimates(POS=[POS_ADJECTIVE, POS_NOUN, POS_VERB])

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 1: 8192 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 2: 4096 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 3: 2048 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 4: 2048 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 5: 2048 distinct words


In [ ]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 3686.4
* Median  of estimates: 2048.0


<font size="+1" color="red">Compute the median of average estimates in 3 separate runs of your algorithm; each run should do 10 passes over the file. Repeat this for nouns (POS_NOUN), adjectives (POS_ADJECTIVE), and verbs (POS_VERB). Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [ ]:
import time
POST=[POS_ADJECTIVE, POS_NOUN, POS_VERB]
for pos in POST:
  start_time = time.time()
  print(f"\nPOS = {pos}:")
  for run in range(3):
      print(f"\nRun {run + 1} :")
      estimates_passes = calculate_estimates(POS=[pos], filename=INPUT_FILE, number_of_passes=10, max_words=10000)
      print(f"Median of estimates: {statistics.median(estimates_passes)}")
      print(f"Average of estimates: {statistics.mean(estimates_passes)}")
  elapsed_time = time.time() - start_time
  print(f"\nTotal time to POS = {pos}: {elapsed_time:.2f} seconds\n")
  print()


POS = JJ:

Run 1 :
- Read 10000/10000 words so far
Estimate on pass 1: 4096 distinct words
- Read 10000/10000 words so far
Estimate on pass 2: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 3: 256 distinct words
- Read 10000/10000 words so far
Estimate on pass 4: 8192 distinct words
- Read 10000/10000 words so far
Estimate on pass 5: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 6: 2048 distinct words
- Read 10000/10000 words so far
Estimate on pass 7: 4096 distinct words
- Read 10000/10000 words so far
Estimate on pass 8: 8192 distinct words
- Read 10000/10000 words so far
Estimate on pass 9: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 10: 16384 distinct words
Median of estimates: 6144.0
Average of estimates: 7705.6

Run 2 :
- Read 10000/10000 words so far
Estimate on pass 1: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 2: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pa

We decrease the max_words because a max_words higher than 10000 waste more than 5 minuts.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>